# Notebook Description

This notebook is used to fetch on chain data by The Graph API of UNISWAP V2. 
<br>Terminology, definition, and example please refer to https://docs.uniswap.org/protocol/V2/reference/API/entities#

# Config Environment

In [1]:
!pip install tqdm

In [1]:
import pandas as pd
import numpy as np
import requests
from pprint import pprint
from tqdm import tqdm

# Query Function

In [2]:

def run_query(q):

    # endpoint where you are making the request
    request = requests.post('https://api.thegraph.com/subgraphs/name/uniswap/uniswap-v2'
                            '',
                            json={'query': q})
    if request.status_code == 200:
        return request.json()
    else:
        raise Exception('Query failed. return code is {}.      {}'.format(request.status_code, q))

def run_query_var(q, var):

    # endpoint where you are making the request
    request = requests.post('https://api.thegraph.com/subgraphs/name/uniswap/uniswap-v2'
                            '',
                            json={'query': q, 'variables': var})
    if request.status_code == 200:
        return request.json()
    else:
        raise Exception('Query failed. return code is {}.      {}'.format(request.status_code, q))


# Uniswap V2 Overview: total liquidity, total volume (ETH/USD), transaction count

In [3]:
v2_overview_query = """
{
 uniswapFactory(id: "0x5C69bEe701ef814a2B6a3EDD4B1652CB9cc5aA6f"){
   totalVolumeUSD
   totalLiquidityUSD
   totalVolumeETH
   totalLiquidityETH
   pairCount
   txCount
 }
}
"""
v2_overview = run_query(v2_overview_query)

pprint(v2_overview)

{'data': {'uniswapFactory': {'pairCount': 76651,
                             'totalLiquidityETH': '908517.0038865886756354051349731031',
                             'totalLiquidityUSD': '1525010933.159837167173990663226338',
                             'totalVolumeETH': '281693120.9858561296889299622023923',
                             'totalVolumeUSD': '413850202508.7221818338680519835316',
                             'txCount': '80559909'}}}


# Get All Pairs (Complete)

In [4]:
# Start from fetching the initial Batch 0: Query 1000 pairs on UNISWAP V2 order by created timestamp
get_paris_query_0 = """
{
  pairs(first: 1000, orderBy: createdAtTimestamp, orderDirection: asc) 
  { 
    token0 {
        id
        symbol
    }
    token1 {
        id
        symbol
    }
    id
    createdAtBlockNumber
    createdAtTimestamp
    totalSupply
    txCount
  }
}
"""

get_pairs_batch0 = run_query(get_paris_query_0)


In [5]:
# Create a dataframe to store the pairs info, initializa with the first 1000 pairs
df_all_pairs = pd.DataFrame.from_dict(get_pairs_batch0['data']['pairs'])

In [6]:
# Do iteration to fetch all the pairs, skip the first 1000 in batch 0
# Start from the last timestamp which is got by batch 0

iter_count = 0
pairs_iter = get_pairs_batch0['data']['pairs']  # Pair info list in each batch
total_pair_amount = len(pairs_iter)  # Initial the total pair amount by 1000 (in batch 0)

# Do loop until 
while len(pairs_iter) == 1000:
  # The last timestamp in the previous query batch
  last_gt = int(pairs_iter[-1]['createdAtTimestamp'])
  params_gt = {"createdAtTimestamp_gt": last_gt}

  # Query 1000 new pairs from the last timestamp
  query_iter = """
  query pairs($createdAtTimestamp_gt: Int!) {
    pairs(first: 1000, orderBy: createdAtTimestamp, orderDirection: asc
    where: {createdAtTimestamp_gt: $createdAtTimestamp_gt }) 
    { 
      token0 {
        id
        symbol
      }
      token1 {
        id
        symbol
      }
      id
      createdAtBlockNumber
      createdAtTimestamp
      totalSupply
      txCount
    }
  }
  """
  result_iter = run_query_var(query_iter, params_gt)

  # List of pair info for this batch
  pairs_iter = result_iter['data']['pairs']

  # Add list of this batch to the dataframe
  df_all_pairs = df_all_pairs.append(pairs_iter, ignore_index=True)

  # Summary for this batch, and update iterator
  iter_count = iter_count + 1
  total_pair_amount = total_pair_amount+len(pairs_iter)
  print('Batch: ', iter_count, '  Length of pair: ', len(pairs_iter), '  Total fected pairs: ', total_pair_amount)
  print('             Start from timestamp: ', last_gt)

print('---------------------------')
print('Amount of fetced pairs: ', total_pair_amount)


/var/folders/sn/bpd3y67s50v2b2bzrzmw_jlc0000gn/T/ipykernel_8615/4199218956.py:42: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_all_pairs = df_all_pairs.append(pairs_iter, ignore_index=True)


Batch:  1   Length of pair:  1000   Total fected pairs:  2000
             Start from timestamp:  1592097373


/var/folders/sn/bpd3y67s50v2b2bzrzmw_jlc0000gn/T/ipykernel_8615/4199218956.py:42: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_all_pairs = df_all_pairs.append(pairs_iter, ignore_index=True)


Batch:  2   Length of pair:  1000   Total fected pairs:  3000
             Start from timestamp:  1594041143


/var/folders/sn/bpd3y67s50v2b2bzrzmw_jlc0000gn/T/ipykernel_8615/4199218956.py:42: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_all_pairs = df_all_pairs.append(pairs_iter, ignore_index=True)


...

/var/folders/sn/bpd3y67s50v2b2bzrzmw_jlc0000gn/T/ipykernel_8615/4199218956.py:42: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_all_pairs = df_all_pairs.append(pairs_iter, ignore_index=True)


Batch:  74   Length of pair:  1000   Total fected pairs:  75000
             Start from timestamp:  1652669173


/var/folders/sn/bpd3y67s50v2b2bzrzmw_jlc0000gn/T/ipykernel_8615/4199218956.py:42: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_all_pairs = df_all_pairs.append(pairs_iter, ignore_index=True)


Batch:  75   Length of pair:  1000   Total fected pairs:  76000
             Start from timestamp:  1653364307
Batch:  76   Length of pair:  651   Total fected pairs:  76651
             Start from timestamp:  1654127704
---------------------------
Amount of fetced pairs:  76651


/var/folders/sn/bpd3y67s50v2b2bzrzmw_jlc0000gn/T/ipykernel_8615/4199218956.py:42: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_all_pairs = df_all_pairs.append(pairs_iter, ignore_index=True)


In [7]:
df_all_pairs

,token0,token1,id,createdAtBlockNumber,createdAtTimestamp,totalSupply,txCount
0,{'id': '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606...,{'id': '0xc02aaa39b223fe8d0a0e5c4f27ead9083c75...,0xb4e16d0168e52d35cacd2c6185b44281ec28c9dc,10008355,1588710145,0.954093348165643525,3547598
1,{'id': '0x8e870d67f660d95d5be530380d0ec0bd3882...,{'id': '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606...,0x3139ffc91b99aa94da8a2dc13f1fc36f9bdc98ee,10008500,1588712092,0.000000006379783,28
2,{'id': '0x06af07097c9eeb7fd685c692751d5c66db49...,{'id': '0xc02aaa39b223fe8d0a0e5c4f27ead9083c75...,0x12ede161c702d1494612d19f05992f43aa6a26fb,10014178,1588788074,14.429278541823799433,825
3,{'id': '0x6b175474e89094c44da98b954eedeac49527...,{'id': '0xc02aaa39b223fe8d0a0e5c4f27ead9083c75...,0xa478c2975ab1ea89e8196811f51a7b7ade33eb11,10042267,1589164213,120131.773005999243656921,1384154
4,{'id': '0x408e41876cccdc0f92210600ef5037265605...,{'id': '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606...,0x07f068ca326a469fc1d87d85d448990c8cba7df9,10060679,1589411030,0.000360555127545398,1450
...,...,...,...,...,...,...,...
76646,{'id': '0xb35a419f98445b8aa2ba98bb4284d332b8b0...,{'id': '0xc02aaa39b223fe8d0a0e5c4f27ead9083c75...,0x81e53f16ef05b6e397b275fca0f54b696d9df665,14939186,1654874855,122.097403879067038627,40
76647,{'id': '0x672e0a50dca95083127a43cade6e8f4fd6db...,{'id': '0xc02aaa39b223fe8d0a0e5c4f27ead9083c75...,0xd98818d62cceef6b478c115e71e28cbb47dd2848,14939199,1654875004,799919.017366845924269339,142
76648,{'id': '0xc02aaa39b223fe8d0a0e5c4f27ead9083c75...,{'id': '0xe00e99280d24865c81a62caa865e89d34938...,0x0d3fc898d438837c1cfc96e1143f9e77574fa2a0,14939226,1654875327,34.465617474213164165,112
76649,{'id': '0x05ddd13a5d05dba84bb4e0fa333e80a5acbd...,{'id': '0xc02aaa39b223fe8d0a0e5c4f27ead9083c75...,0xde010722eeef0f3aae3524dbf094b4b1551acf06,14939401,1654877773,13.564659966250535278,87


In [ ]:
# Write dataframe to csv
# The information of all pairs on UNISWAP V2 until 10/06/2022 UTC 17:20
df_all_pairs.to_csv("PoolDataV2/All_Pairs_Info_V2_20220610.csv")

## TODO: Try to label the untracked pair

# Count Mints/Burns Transactions (Incomplete)

In [10]:
# # Mint: Add Liquidity to pool,  Burn: Remove Liquidity to the pool
# # See example of Add ETH and RND to pair: ETH/RND 0x5449bd1a97296125252db2d9cf23d5d6e30ca3c1
# mint_transaction_query = """
# {
#  mints(first: 30, where: { pair: "0x5449bd1a97296125252db2d9cf23d5d6e30ca3c1" }, orderBy: timestamp, orderDirection: desc) {
#    transaction {
#      id
#      timestamp
#    }
#    to
#    liquidity
#    amount0
#    amount1
#    amountUSD
#  }
# }
# """
# mint_transaction = run_query(mint_transaction_query)

# pprint(mint_transaction)

candidatePairs = [
 "0xa478c2975ab1ea89e8196811f51a7b7ade33eb11",
 "0xae461ca67b15dc8dc81ce7615e0320da1a9ab8d5"
]
params_candidate_pairs = {"candidatePairs": candidatePairs}
mint_transaction_query = """
query($candidatePairs: [String!]) {
 mints(first: 30, where: { pair_in: $candidatePairs }, orderBy: timestamp, orderDirection: desc) {
   transaction {
     id
     timestamp
   }
   to
   liquidity
   amount0
   amount1
   amountUSD
 }
}
"""
mint_transaction = run_query_var(mint_transaction_query, params_candidate_pairs)

pprint(mint_transaction)

{'data': {'mints': [{'amount0': '499.7',
                     'amount1': '500.323928',
                     'amountUSD': '999.2019265940661032058999889692367',
                     'liquidity': '0.000444144308078184',
                     'to': '0xd2911a916d6c8ce109d2fb7c7e38f10ffd1f4dcc',
                     'transaction': {'id': '0x9e9e281309ca604795cc177ee273082c8a82b09464373b4574e05816a32d64cf',
                                     'timestamp': '1654700992'}},
                    {'amount0': '4.999474616458787883',
                     'amount1': '0.002752486511869076',
                     'amountUSD': '10.02812481062376138788770791438522',
                     'liquidity': '0.069863367895800931',
                     'to': '0x8fe952be73de98f3e364149f80174df6739ceed2',
                     'transaction': {'id': '0x666899c773a26a8d81001a988c4e3fb442f1cef1dc8bf1bfac5c9e7e5287d866',
                                     'timestamp': '1654652158'}},
                    {'amount0': '20

# Compute Volume (Incomplete)

In [11]:
# query current USD price of ETH in Uniswap based on a weighted average of stablecoins.
eth_price_query = """
{
 bundle(id: "1") {
   ethPrice
 }
}
"""
eth_price = run_query(eth_price_query)

pprint(eth_price)

{'data': {'bundle': {'ethPrice': '1679.054792895824716182178552301571'}}}


In [12]:
# query current USD price of ETH in Uniswap based on a weighted average of stablecoins.
eth_price_pairs = [
 "0xb4e16d0168e52d35cacd2c6185b44281ec28c9dc"
]
params_eth_price_pairs = {"eth_price_pairs": eth_price_pairs}
eth_price_byvolume_query = """
query($eth_price_pairs: [String!]) {
 swaps(first: 1, 
  where: { pair_in: $eth_price_pairs, timestamp_gt: 1654503528 }, orderBy: timestamp, orderDirection: asc) {
   transaction {
     id
     timestamp
   }
   amount0In
   amount1In
   amount0Out
   amount1Out
   amountUSD
 }
}
"""
eth_price_byvolume = run_query_var(eth_price_byvolume_query, params_eth_price_pairs)

pprint(eth_price_byvolume)

if float(eth_price_byvolume['data']['swaps'][0]['amount0In']) != 0:
  eth_price_byvolume_result = float(eth_price_byvolume['data']['swaps'][0]['amount0In'])/float(eth_price_byvolume['data']['swaps'][0]['amount1Out'])
else:
  eth_price_byvolume_result = float(eth_price_byvolume['data']['swaps'][0]['amount0Out'])/float(eth_price_byvolume['data']['swaps'][0]['amount1In'])

print('ETH PRICE IS: ', eth_price_byvolume_result)

{'data': {'swaps': [{'amount0In': '539.626088',
                     'amount0Out': '0',
                     'amount1In': '0',
                     'amount1Out': '0.284165115514204348',
                     'amountUSD': '538.676591762887036143561404921746',
                     'transaction': {'id': '0xea9d52361194f848356746a2182965d0419138ff170e67fea13a2803fc464fef',
                                     'timestamp': '1654503529'}}]}}
ETH PRICE IS:  1898.9878015939155


In [13]:
print(2634.665349/1.472669921217362735)
print(167.37499/0.093561282132463456)

print(1898.9878015939155*0.016)

1789.0399681838337
1788.9343346431656
30.383804825502647


In [14]:
# Type 1: trade against WETH
pair_daily_volume_query = """
{
swaps(orderBy: timestamp, orderDirection: desc, where:
 { pair: "0xc9919a9eb691f571e41db417f911135b4ac46fe7" }
) {
     transaction {
        id
        timestamp
      }
     pair {
       token0 {
         symbol
       }
       token1 {
         symbol
       }
     }
     id
     transaction
     amount0In
     amount0Out
     amount1In
     amount1Out
     amountUSD
     to
 }
}
"""
pair_daily_volume = run_query(pair_daily_volume_query)

pprint(pair_daily_volume)

{'data': {'swaps': [{'amount0In': '0',
                     'amount0Out': '954.10490882867114783',
                     'amount1In': '0.087109307299713778',
                     'amount1Out': '0',
                     'amountUSD': '162.3017323177359330773202940380697',
                     'id': '0xe976f2ebd70135153e8522c3aaa3c9d489717ef621d69b9e00a9f215d0474918-8',
                     'pair': {'token0': {'symbol': 'LSS'},
                              'token1': {'symbol': 'WETH'}},
                     'to': '0x62e3c242b5e903071458ad90a160493d84911c77',
                     'transaction': {'id': '0xe976f2ebd70135153e8522c3aaa3c9d489717ef621d69b9e00a9f215d0474918',
                                     'timestamp': '1654768688'}},
                    {'amount0In': '649.3506494',
                     'amount0Out': '0',
                     'amount1In': '0',
                     'amount1Out': '0.057907220009412727',
                     'amountUSD': '102.5315103855514293805263468787294',

# Top 50 most volume pairs (undirectional)

In [15]:
# Take example of 2022/06/05 0:00:00 (GMT+0)  timestamp: 1654387200
most_volume_pairs_query = """
{
 pairDayDatas(first: 50, orderBy: dailyVolumeUSD, orderDirection: desc,
   where: {
     date: 1654387200
   }
 ) {
     pairAddress
     token0
     token1
     reserve0
     reserve1
     reserveUSD
     totalSupply
     dailyTxns
     dailyVolumeToken0
     dailyVolumeToken1
     dailyVolumeUSD
 }
}
"""
most_volume_pairs = run_query(most_volume_pairs_query)


In [16]:
# Create a dataframe to store the popular pairs by daily volume
df_most_volume_pairs = pd.DataFrame.from_dict(most_volume_pairs['data']['pairDayDatas'])
df_most_volume_pairs

,pairAddress,reserve0,reserve1,reserveUSD,totalSupply,dailyTxns,dailyVolumeToken0,dailyVolumeToken1,dailyVolumeUSD
0,0x26b1c843b41ab3ac6146f859d31dcb2031f5992b,1.767872165974808497,0.103575127241738334,185.007627282511291050410476253084,0.397489622506046319,66,75.590458431395684271,20391.289551617179340048,18044577.16616131095138411684391161
1,0x9112d1761124d4ec7aca02014d6e215970f87e33,16350613868.3869050774532976,0.519906171224847849,939.5606044999527414057226528192348,76481.596984279553163924,568,1419137291173.507923209293317101,13969.023979241586925419,12330161.8603183632501027905931409
2,0x7a809081f991ecfe0ab2727c7e90d2ad7c2e411e,7278284.442263,53711361613298.579569450654648315,14562523.79614463010296044124735658,17000.372001445604539439,1130,9352341.202544,70352106872638.631096732439854899,9356068.767914834418701521183459145
3,0xb4e16d0168e52d35cacd2c6185b44281ec28c9dc,79053219.45668,43762.070870754729340122,158171118.2947124454132655087211276,0.949291097224270471,2352,7944060.293021,4416.753923820066502526,7946290.543183526065581561379215337
4,0x0d4a11d5eeaac28ec3f61d100daf4d40471f1852,17150.6216251843007197,31025675.01265,61986741.93124518014117041922362383,0.358735792422925442,988,1985.069246836202839412,3568632.914985,3567514.616175831645067170006679848
5,0x6033368e4a402605294c91cf5c03d72bd96e7d8d,12923069.47957958366364951,1700.778808333284489473,6150046.731936887471116457848148126,122495.841175114799947429,339,13267781.010060489687022394,1890.06115884505521772,3406788.640697228554984176713442879
6,0x5fc5b2a2a84e577e8eb3ddd00d6b18cdfc976645,1.405332611560299561,4879.509395835,2540.637390630996495380319988943423,0.000000001693876534,52,2370.283509828503112744,29395.18460405,2117245.547362322563131098884959673
7,0xc9919a9eb691f571e41db417f911135b4ac46fe7,19492.596085670694340666,1.743545235299297085,6230.195069012601572355339755270908,134.786530428602600527,5,39890.453065187663700397,2315.4491801142244733,2068207.090312683960461532632688075
8,0x55d5c232d921b9eaa6b37b5845e439acd04b4dba,9561300.85832576,262.860789083706771544,950046.2579647230507226365979986013,0.141939588833336829,1084,31687521.75123634,921.849962520731810018,1656077.459784708517237385640827616
9,0x5281e311734869c64ca60ef047fd87759397efe6,1590.131078037393753415,194274555743.182871721195811459,5747440.852724872320382470833962709,13027895.889162734975983901,354,693.44156563152035369,84402864787.057603459657130526,1242412.424525914084718022581442596


In [17]:
# Create a data frame of orders
df_top50_pairs_info = pd.DataFrame({'pairAddress': df_most_volume_pairs['pairAddress'], 'dailyVolumeUSD': df_most_volume_pairs['dailyVolumeUSD'], 'token0_symbol': np.nan, 'token0_id': np.nan, 'dailyVolumeToken0': df_most_volume_pairs['dailyVolumeToken0'], 'reserve0': df_most_volume_pairs['reserve0'], 'token1_symbol': np.nan, 'token1_id': np.nan, 'dailyVolumeToken1': df_most_volume_pairs['dailyVolumeToken1'], 'reserve1': df_most_volume_pairs['reserve1'], 'reserveUSD': df_most_volume_pairs['reserveUSD'], 'totalSupply': df_most_volume_pairs['totalSupply'], 'dailyTxns': df_most_volume_pairs['dailyTxns'], }, index=df_most_volume_pairs.index)

for index, row in tqdm(df_most_volume_pairs.iterrows(), total=df_most_volume_pairs.shape[0]):
  this_id = row['pairAddress']  # Pair address of this row
  params_pair_id = {"id": this_id}
  pair_info_query = """
    query pair($id: String!) 
    {
      pair(id: $id){
      token0 {
        id
        symbol
      }
      token1 {
        id
        symbol
      }
      }
    }
  """
  pair_info = run_query_var(pair_info_query, params_pair_id)
  df_top50_pairs_info.loc[index] = [row['pairAddress'], row['dailyVolumeUSD'], pair_info['data']['pair']['token0']['symbol'], pair_info['data']['pair']['token0']['id'], row['dailyVolumeToken0'], row['reserve0'], pair_info['data']['pair']['token1']['symbol'], pair_info['data']['pair']['token0']['id'], row['dailyVolumeToken1'], row['reserve1'], row['reserveUSD'], row['totalSupply'], row['dailyTxns']]

100%|███████████████████████████████████████████| 50/50 [00:14<00:00,  3.39it/s]


In [18]:
df_top50_pairs_info

,pairAddress,dailyVolumeUSD,token0_symbol,token0_id,dailyVolumeToken0,reserve0,token1_symbol,token1_id,dailyVolumeToken1,reserve1,reserveUSD,totalSupply,dailyTxns
0,0x26b1c843b41ab3ac6146f859d31dcb2031f5992b,18044577.16616131095138411684391161,sILV2,0x7e77dcb127f99ece88230a64db8d595f31f1b068,75.590458431395684271,1.767872165974808497,WETH,0x7e77dcb127f99ece88230a64db8d595f31f1b068,20391.289551617179340048,0.103575127241738334,185.007627282511291050410476253084,0.397489622506046319,66
1,0x9112d1761124d4ec7aca02014d6e215970f87e33,12330161.8603183632501027905931409,SHIT,0x4e4a47cac6a28a62dcc20990ed2cda9bc659469f,1419137291173.507923209293317101,16350613868.3869050774532976,WETH,0x4e4a47cac6a28a62dcc20990ed2cda9bc659469f,13969.023979241586925419,0.519906171224847849,939.5606044999527414057226528192348,76481.596984279553163924,568
2,0x7a809081f991ecfe0ab2727c7e90d2ad7c2e411e,9356068.767914834418701521183459145,USDC,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,9352341.202544,7278284.442263,CAW,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,70352106872638.631096732439854899,53711361613298.579569450654648315,14562523.79614463010296044124735658,17000.372001445604539439,1130
3,0xb4e16d0168e52d35cacd2c6185b44281ec28c9dc,7946290.543183526065581561379215337,USDC,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,7944060.293021,79053219.45668,WETH,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,4416.753923820066502526,43762.070870754729340122,158171118.2947124454132655087211276,0.949291097224270471,2352
4,0x0d4a11d5eeaac28ec3f61d100daf4d40471f1852,3567514.616175831645067170006679848,WETH,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,1985.069246836202839412,17150.6216251843007197,USDT,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,3568632.914985,31025675.01265,61986741.93124518014117041922362383,0.358735792422925442,988
5,0x6033368e4a402605294c91cf5c03d72bd96e7d8d,3406788.640697228554984176713442879,X2Y2,0x1e4ede388cbc9f4b5c79681b7f94d36a11abebc9,13267781.010060489687022394,12923069.47957958366364951,WETH,0x1e4ede388cbc9f4b5c79681b7f94d36a11abebc9,1890.06115884505521772,1700.778808333284489473,6150046.731936887471116457848148126,122495.841175114799947429,339
6,0x5fc5b2a2a84e577e8eb3ddd00d6b18cdfc976645,2117245.547362322563131098884959673,WETH,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,2370.283509828503112744,1.405332611560299561,GIZA,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,29395.18460405,4879.509395835,2540.637390630996495380319988943423,0.000000001693876534,52
7,0xc9919a9eb691f571e41db417f911135b4ac46fe7,2068207.090312683960461532632688075,LSS,0x3b9be07d622accaed78f479bc0edabfd6397e320,39890.453065187663700397,19492.596085670694340666,WETH,0x3b9be07d622accaed78f479bc0edabfd6397e320,2315.4491801142244733,1.743545235299297085,6230.195069012601572355339755270908,134.786530428602600527,5
8,0x55d5c232d921b9eaa6b37b5845e439acd04b4dba,1656077.459784708517237385640827616,HEX,0x2b591e99afe9f32eaa6214f7b7629768c40eeb39,31687521.75123634,9561300.85832576,WETH,0x2b591e99afe9f32eaa6214f7b7629768c40eeb39,921.849962520731810018,262.860789083706771544,950046.2579647230507226365979986013,0.141939588833336829,1084
9,0x5281e311734869c64ca60ef047fd87759397efe6,1242412.424525914084718022581442596,WETH,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,693.44156563152035369,1590.131078037393753415,CULT,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,84402864787.057603459657130526,194274555743.182871721195811459,5747440.852724872320382470833962709,13027895.889162734975983901,354


In [19]:
# Write dataframe to csv
# Daily aggregated data: undirectional volume USD on 05/06/2022
df_top50_pairs_info.to_csv("PoolDataV2/Top_50_Volume_Pairs_Undirectional_20220605.csv")

# Compute Directional Volume (Incomplete)

In [20]:
# Firstly, fetch all transaction in each day
directional_volume_pair = [
 "0xc9919a9eb691f571e41db417f911135b4ac46fe7"
]
params_directional_volume_pair = {"directional_volume_pair": directional_volume_pair}

swaps_transaction_query = """
query($directional_volume_pair: [String!]) {
 swaps(first: 5, where: { pair_in: $directional_volume_pair, timestamp_gt: 1654387200}, orderBy: timestamp, orderDirection: asc) {
   transaction {
     id
     timestamp
   }
   amount0In
   amount1In
   amount0Out
   amount1Out
   amountUSD
 }
}
"""
swaps_transaction = run_query_var(swaps_transaction_query, params_directional_volume_pair)

pprint(swaps_transaction)

{'data': {'swaps': [{'amount0In': '874.358028061008428415',
                     'amount0Out': '0',
                     'amount1In': '0',
                     'amount1Out': '0.080380204908462933',
                     'amountUSD': '140.9912911183920638860407598128405',
                     'transaction': {'id': '0x6a8031402df1401112a7b5a17654673c708d111912946ac1beb169a13d506f59',
                                     'timestamp': '1654417560'}},
                    {'amount0In': '0',
                     'amount0Out': '384.915147460101858337',
                     'amount1In': '0.03469375',
                     'amount1Out': '0',
                     'amountUSD': '62.32797447381097606245121727947374',
                     'transaction': {'id': '0xa3a509d48b4c9c6432ffd0c46cac6e6d1fad5395a4cf1dcaa0fb6dd98d1a7b82',
                                     'timestamp': '1654426696'}},
                    {'amount0In': '0',
                     'amount0Out': '19167.809153296665628187',
        

# **Other Data and Supporting Resources**

## Token Overview (Supporting Resources)

Token data can be fetched using the token contract address as an ID. Token data is aggregated across all pairs the token is included in. Any token that is included in some pair in Uniswap can be queried.
<br>reference: https://docs.uniswap.org/protocol/V2/reference/API/queries

In [21]:
# Example: The allPairs field gets the first 200 pairs DAI is included in sorted by liquidity in derived USD
token_overview_query = """
{
 token(id: "0x6b175474e89094c44da98b954eedeac495271d0f"){
   name
   symbol
   decimals
   derivedETH
   tradeVolumeUSD
   totalLiquidity
 }
}
"""
token_overview = run_query(token_overview_query)

pprint(token_overview)

{'data': {'token': {'decimals': '18',
                    'derivedETH': '0.0005962864061262205504427291717104448',
                    'name': 'Dai Stablecoin',
                    'symbol': 'DAI',
                    'totalLiquidity': '81822471.521936713104024906',
                    'tradeVolumeUSD': '19194080355.67736561812158445797713'}}}


The allPairs field gets the first 200 pairs DAI is included in sorted by liquidity in derived USD.
<br>The result of `total liquidity` is consistent with https://v2.info.uniswap.org/token/0x6b175474e89094c44da98b954eedeac495271d0f


# All Token in Uniswap V2 (Complete)

In [22]:
# Start from fetching the initial Batch 0: Query 1000 tokens on UNISWAP V2 order by id (asc)
# tradeVolumeUSD: amount of token in USD traded all time across pairs (only for tokens with liquidity above minimum threshold)
# untrackedVolumeUSD: amount of token in USD traded all time across pairs (no minimum liquidity threshold)
get_all_tokens_batch0_query = """
{
   tokens(first: 1000, orderBy: id, orderDirection: asc) {
     id
     name
     symbol
     tradeVolume
     tradeVolumeUSD
     untrackedVolumeUSD
     txCount
     totalLiquidity
     derivedETH
   }
}
"""
get_all_tokens_batch0 = run_query(get_all_tokens_batch0_query)

In [23]:
# Create a dataframe to store the tokens info, initializa with the first 1000 tokens
df_all_tokens = pd.DataFrame.from_dict(get_all_tokens_batch0['data']['tokens'])

In [25]:
# Do iteration to fetch all the tokens, skip the first 1000 in batch 0
# Start from the last id which is got by batch 0

tokens_iter_count = 0
tokens_iter = get_all_tokens_batch0['data']['tokens']  # Token info list in each batch
total_tokens_amount = len(tokens_iter)  # Initial the total token amount by 1000 (in batch 0)

# Do loop until 
while len(tokens_iter) == 1000:
  # The last id in the previous query batch
  last_id_gt = str(tokens_iter[-1]['id'])
  params_id_gt = {"id_in": last_id_gt}

  # Query 1000 new tokens from the last id
  query_iter = """
  query tokens($id_in: String!) {
    tokens(first: 1000, orderBy: id, orderDirection: asc
    where: {id_gt: $id_in }) 
    { 
      id
      name
      symbol
      tradeVolume
      tradeVolumeUSD
      untrackedVolumeUSD
      txCount
      totalLiquidity
      derivedETH
    }
  }
  """
  tokens_result_iter = run_query_var(query_iter, params_id_gt)

  # List of token info for this batch
  tokens_iter = tokens_result_iter['data']['tokens']

  # Add list of this batch to the dataframe
  df_all_tokens = df_all_tokens.append(tokens_iter, ignore_index=True)

  # Summary for this batch, and update iterator
  tokens_iter_count = tokens_iter_count + 1
  total_tokens_amount = total_tokens_amount+len(tokens_iter)
  print('Batch: ', tokens_iter_count, '  Length of pair: ', len(tokens_iter), '  Total fected pairs: ', total_tokens_amount)
  print('             Start from id: ', last_id_gt)

print('---------------------------')
print('Amount of fetced pairs: ', total_tokens_amount)


/var/folders/sn/bpd3y67s50v2b2bzrzmw_jlc0000gn/T/ipykernel_8615/1163013731.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_all_tokens = df_all_tokens.append(tokens_iter, ignore_index=True)


Batch:  1   Length of pair:  1000   Total fected pairs:  2000
             Start from id:  0x03703a5e60918059baf5481c5d55a553890207a2


/var/folders/sn/bpd3y67s50v2b2bzrzmw_jlc0000gn/T/ipykernel_8615/1163013731.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_all_tokens = df_all_tokens.append(tokens_iter, ignore_index=True)


Batch:  2   Length of pair:  1000   Total fected pairs:  3000
             Start from id:  0x06f3c323f0238c72bf35011071f2b5b7f43a054c


/var/folders/sn/bpd3y67s50v2b2bzrzmw_jlc0000gn/T/ipykernel_8615/1163013731.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_all_tokens = df_all_tokens.append(tokens_iter, ignore_index=True)


...

/var/folders/sn/bpd3y67s50v2b2bzrzmw_jlc0000gn/T/ipykernel_8615/1163013731.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_all_tokens = df_all_tokens.append(tokens_iter, ignore_index=True)


Batch:  69   Length of pair:  1000   Total fected pairs:  70000
             Start from id:  0xf7327beb21bc9a1c3af5523f20162f857ec3fc41


/var/folders/sn/bpd3y67s50v2b2bzrzmw_jlc0000gn/T/ipykernel_8615/1163013731.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_all_tokens = df_all_tokens.append(tokens_iter, ignore_index=True)


Batch:  70   Length of pair:  1000   Total fected pairs:  71000
             Start from id:  0xfb0e7ca6b64eac2c58a757d1359291c35f211b5b
Batch:  71   Length of pair:  284   Total fected pairs:  71284
             Start from id:  0xfeee01039cdface170e547237ddd64f28a398f7e
---------------------------
Amount of fetced pairs:  71284


/var/folders/sn/bpd3y67s50v2b2bzrzmw_jlc0000gn/T/ipykernel_8615/1163013731.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_all_tokens = df_all_tokens.append(tokens_iter, ignore_index=True)


In [26]:
df_all_tokens

,id,name,symbol,tradeVolume,tradeVolumeUSD,untrackedVolumeUSD,txCount,totalLiquidity,derivedETH
0,0x00000000000045166c45af0fc6e4cf31d9e14b9a,TopBidder,BID,36520506.79045913892332785,23362640.36887859622558915387398381,23466279.70596361174072586983085042,2706,2252392.839917663338975365,0.00010395165265860908001061772755218
1,0x0000000000004946c0e9f43f4dee607b0ef1fa1c,Chi Gastoken by 1inch,CHI,43580282,19360715.70503782693540014871432548,65455773.59876190292325552554250769,75957,23374,0
2,0x000000000000d0151e748d25b766e77efe2a6c83,XDEFI Governance Token,XDEX,63890993.176575438849139776,18713723.91429208465798213093796374,18977868.34193645649430618371779554,4400,7459590.572861402421977784,0.00001160462895444170070268104491209126
3,0x0000000000085d4780b73119b644ae5ecd22b376,TrueUSD,TUSD,184555552.794689017276039881,176239741.7006048194469590153693985,185841110.6556808365070216844143068,47059,46910.632058833121697624,0.0005872267247000947115965131345753764
4,0x0000000000095413afc295d19edeb1ad7b71c952,Tokenlon,LON,142792553.719051369295010464,518261039.9061991068052100818991129,520112755.1339709640673245365510702,57987,1931446.73103393616330934,0.0004367451890548845771272978402901951
...,...,...,...,...,...,...,...,...,...
91279,0xfffed8edc3ddc66de57f7fea41f07f98595f0a5c,TradeStars,TSX,0,0,0,2,0.00000000000013239,0.00006042752473751793942140645063826573
91280,0xffff49dc8ed20043fb9e5d65ea2f3e071958df8e,Alaskan Malamute,MALA,21789078739.712294975,0,5393.769723343128401960611013309801,7,0.000002959,0.0000000001145657316661034133153092260898952
91281,0xfffff4ff386013f4b9dac8874f856dec0e802467,Moon Doge,MOGE,19150445234.85529503422301082,0,1173.552612430322387524868188060823,8,0.000000000180785101,0.000000005531429274141346415488077195033898
91282,0xffffffff2ba8f66d4e51811c5190992176930278,Furucombo,COMBO,130417880.769097013663764675,185309346.3696076804844877325321709,185516831.2283832158632967962866492,23842,2061453.687712022160325331,0.00002902950409227019807739813744618048


In [27]:
# Write dataframe to csv
# The information of all tokens until 10/06/2022 UTC 17:20
df_all_tokens.to_csv("PoolDataV2/All_Tokens_Info_V2_20220610.csv")

# Get top 200 popular tokens (order by the total trade volume USD)

In [28]:
# Get the top 200 popular tokens order by the total trade volume USD
get_popular_tokens_query = """
{
   tokens(first: 200, orderBy: tradeVolumeUSD, orderDirection: desc) {
     id
     name
     symbol
     tradeVolumeUSD
   }
}
"""
get_popular_tokens = run_query(get_popular_tokens_query)

In [29]:
# Create a dataframe to store the popular tokens
df_popular_tokens = pd.DataFrame.from_dict(get_popular_tokens['data']['tokens'])
df_popular_tokens

,id,name,symbol,tradeVolumeUSD
0,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,Wrapped Ether,WETH,387877943993.2650258550199180086558
1,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,USD//C,USDC,57151087088.85927425234764115638086
2,0xdac17f958d2ee523a2206206994597c13d831ec7,Tether USD,USDT,46643732924.40661365246692484776756
3,0x6b175474e89094c44da98b954eedeac495271d0f,Dai Stablecoin,DAI,19194080457.96787237191109652535303
4,0x05934eba98486693aaec2d00b0e9ce918e37dc3f,You don't blacklist delta.financial,SCAMMY,11029715648.72159798743878461135205
...,...,...,...,...
195,0xeec2be5c91ae7f8a338e1e5f3b5de49d07afdc81,Dopex Governance Token,DPX,287878518.4337276001149145019920483
196,0x0d8775f648430679a709e98d2b0cb6250d2887ef,Basic Attention Token,BAT,287655138.7316011364479310256153892
197,0x3155ba85d5f96b2d030a4966af206230e46849cb,THORChain ETH.RUNE,RUNE,287355918.9337234532037635607585532
198,0xee573a945b01b788b9287ce062a0cfc15be9fd86,Exeedme,XED,286489974.8979467423185122250284526


In [30]:
# Write dataframe to csv
# Order by the historical trade volume USD until 10/06/2022 UTC 17:20
df_popular_tokens.to_csv("PoolDataV2/Top200_Volume_Tokens_V2_20220610.csv")